In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [3]:
### Scraping Instagram for urls, likes, and  via Selenium ###
username='manofthekinfolk'
account='natgeo'
browser = webdriver.Chrome(executable_path='C:\\Users\\david\\Downloads\\chromedriver\\chromedriver.exe')
browser.get('https://www.instagram.com/'+account+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 1. Login (Burner Account
# 2. Scroll down for several minutes
# 3. Run code below

In [29]:
number_of_images = 10000
browser = webdriver.Chrome(executable_path='C:\\Users\\david\\Downloads\\chromedriver\\chromedriver.exe')
browser.get('https://www.instagram.com/'+account+'/?hl=en')
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
links=[]
while len(links) < number_of_images:
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
            pre_link = links.append('https://www.instagram.com'+link.get('href'))
            if pre_link not in links:
                links.append(pre_link)
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight/4.0);")
        time.sleep(5)
        new_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if last_height == new_height:
            break
        last_height = new_height
#     return links
        


In [30]:
len(links)

10021

In [31]:
result=pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
        x.columns =  x.columns.str.replace("shortcode_media.", "")
        result=result.append(x)
       
    except:
        np.nan
#Just check for the duplicates
result = result.drop_duplicates(subset = 'shortcode')
result.index = range(len(result.index))

In [32]:
result2 = pd.concat([result.loc[:,'display_url'],result.loc[:,'edge_media_to_caption.edges'],result.loc[:,'edge_media_to_parent_comment.count'],result.loc[:,'edge_media_preview_like.count']], axis='columns')
result2 = result2.rename(columns={"display_url":"URL","edge_media_to_caption.edges":"UnfinishedCaption","edge_media_to_parent_comment.count":"Comment Count","edge_media_preview_like.count":"Like Count"})
result2['UnfinishedCaption']=result2['UnfinishedCaption'].astype(str)
result2.loc[:, 'Caption'] = result2.UnfinishedCaption.map(lambda x: x[20:-4])
result2 = pd.concat([result2.loc[:,'URL'],result2.loc[:,'Caption'],result2.loc[:,'Like Count'],result2.loc[:,'Comment Count']], axis='columns')
word = 'Video'   
result3 = result2[~result2["Caption"].str.contains(word, na=False)]
word = 'Photos'
result4 = result3[~result3["Caption"].str.contains(word, na=False)]
result4

,URL,Caption,Like Count,Comment Count
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
6,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nichole Sobecki @nicholesobecki | Omm...,114367,297
7,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
...,...,...,...,...
1087,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
1092,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
1093,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
1095,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


In [33]:
export_csv = result4.to_csv (r'C:\Users\david\Pictures\UGCA\Final\Instagram_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [35]:
unedited_csv=result.to_csv (r'C:\Users\david\Pictures\UGCA\Final\Unedited_Instagram_dataframe.csv', index = None, header=True)

In [36]:
result4.index = range(len(result4.index))
result4

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nichole Sobecki @nicholesobecki | Omm...,114367,297
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
...,...,...,...,...
952,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
953,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
954,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
955,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


### Importing data from csv to Dataframe

In [18]:
full = pd.read_csv(r'C:\Users\david\Pictures\UGCA\Final\Instagram_dataframe.csv')

### Test data set 

In [19]:
full = pd.concat([full.loc[0:2,:],full.loc[4:188,:],full.loc[275:340,:],full.loc[341:381,:],full.loc[383:558,:],full.loc[560:672,:],full.loc[674:695,:],full.loc[697:705,:],full.loc[707:774,:],full.loc[776:870,:],full.loc[872:945,:],full.loc[947:,:]])
full.index = range(len(full.index))
full

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @gerdludwig | Known for their elabora...,149658,394
...,...,...,...,...
857,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
858,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
859,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
860,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


### Actual Dataset 

In [20]:
api_df = full.loc[0:538,:]
api_df = pd.concat([full.loc[0:6,:],full.loc[12:186,:],full.loc[193:250,:],full.loc[254:290,:],full.loc[295:466,:],full.loc[491:541,:]])
api_df

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @gerdludwig | Known for their elabora...,149658,394
...,...,...,...,...
537,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Ira Block @irablockphoto | A red-tail...,454383,1601
538,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nora Lorek @noralorek | Three years a...,122497,377
539,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,"Photo by Ami Vitale @amivitale | On Monday, @U...",493142,2411
540,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Paolo Woods and Gabriele Galimberti ...,331186,2307


In [21]:
_csv=api_df.to_csv (r'C:\Users\david\Pictures\UGCA\finding_6.csv', index = None, header=True)

# Google Vision API


In [22]:
list_of_urls = api_df["URL"].tolist() #change test to api_df when ready

In [23]:
from google.cloud import vision
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keyFile.json"
image_urls = list_of_urls
ls_of_label_for_url = []

for url in image_urls:
    image_uri = url
    label_list = []
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = image_uri
    response = client.label_detection(image=image)
    print('Labels (and confidence score):')
    print('=' * 79)
    
    for label in response.label_annotations:
        label_list.append(label.description)
        print(f'{label.description} ({label.score*100.:.2f}%)')
    ls_of_label_for_url.append(label_list)
    
print(ls_of_label_for_url)
print('Length of unedited list',len(ls_of_label_for_url))

ls_of_label_for_url = list(filter(None, ls_of_label_for_url))
print(ls_of_label_for_url)
print('Length of edited list', len(ls_of_label_for_url))

Labels (and confidence score):
Rays and skates (88.14%)
Fish (82.69%)
Underwater (81.39%)
Water (80.73%)
Manta ray (77.54%)
Organism (76.72%)
Marine biology (73.21%)
Stingray (71.06%)
Cartilaginous fish (61.89%)
Skate (60.42%)
Labels (and confidence score):
Photography (62.42%)
Labels (and confidence score):
Badlands (98.23%)
Mountainous landforms (98.08%)
Sky (97.49%)
Nature (95.35%)
Mountain (93.90%)
Rock (93.53%)
Formation (91.75%)
Cloud (91.28%)
Wilderness (89.94%)
Geology (88.88%)
Labels (and confidence score):
Rock (69.81%)
Tourism (69.71%)
Landscape (59.31%)
National park (54.88%)
Labels (and confidence score):
People (95.68%)
Community (84.84%)
Yellow (83.63%)
Tradition (76.61%)
Event (76.34%)
Temple (72.84%)
Tourism (68.09%)
Adaptation (66.97%)
Sari (66.58%)
Smile (64.11%)
Labels (and confidence score):
Water (95.78%)
Photograph (95.59%)
Sky (94.83%)
Sea (93.78%)
Skin (90.60%)
Beauty (90.38%)
Ocean (87.65%)
Vacation (86.62%)
Summer (83.16%)
Cloud (80.34%)
Labels (and confidenc

Labels (and confidence score):
Sky (97.45%)
Water (95.13%)
Blue (92.70%)
Sea (85.70%)
Atmosphere (84.04%)
Cloud (83.81%)
Ocean (83.12%)
Horizon (70.93%)
Rock (69.81%)
Tree (68.69%)
Labels (and confidence score):
Sky (99.05%)
Cloud (98.27%)
White (97.18%)
Black (97.01%)
Black-and-white (95.77%)
Nature (95.50%)
Monochrome photography (95.11%)
Monochrome (93.93%)
Water (91.00%)
Natural landscape (88.55%)
Labels (and confidence score):
Yellow (86.49%)
Sky (82.21%)
Sand (73.38%)
Dune (66.73%)
Landscape (65.04%)
Heat (51.50%)
Aeolian landform (51.19%)
Labels (and confidence score):
Design (72.03%)
Room (71.39%)
Photography (70.60%)
Material property (69.92%)
Technology (67.58%)
Architecture (65.45%)
Black-and-white (56.41%)
Collage (55.30%)
Interior design (54.31%)
Art (50.22%)
Labels (and confidence score):
Room (82.12%)
Photography (62.42%)
Sitting (60.23%)
Labels (and confidence score):
Art (50.22%)
Labels (and confidence score):
People (95.89%)
Green (94.52%)
Product (93.48%)
Social grou

Labels (and confidence score):
Wildlife (97.06%)
Hyena (90.16%)
Grassland (88.67%)
Natural landscape (88.55%)
Savanna (88.54%)
Spotted hyena (88.37%)
Safari (84.21%)
Plain (78.23%)
Lion (77.64%)
Ecoregion (75.62%)
Labels (and confidence score):
Snow (96.48%)
Winter (95.78%)
Bridge (91.46%)
Trestle (84.69%)
Freezing (83.59%)
Water (80.73%)
Wood (69.17%)
Landscape (65.04%)
Pier (64.72%)
Mountain (54.44%)
Labels (and confidence score):
Blue (98.02%)
Sky (95.85%)
Eyewear (88.57%)
House (85.83%)
Glasses (84.64%)
Fun (82.20%)
Winter (82.14%)
Cloud (76.76%)
Mountain (73.80%)
Home (68.23%)
Labels (and confidence score):
Yellow (83.63%)
Watercolor paint (73.25%)
Art (67.73%)
Visual arts (64.93%)
Paint (55.47%)
Painting (53.36%)
Illustration (50.41%)
Labels (and confidence score):
Child (86.28%)
Adaptation (81.25%)
Photography (62.42%)
Sitting (57.49%)
Tribe (56.67%)
Smile (54.04%)
Art (50.22%)
Labels (and confidence score):
Loggerhead sea turtle (98.05%)
Sea turtle (97.95%)
Green sea turtle (96

Labels (and confidence score):
Grassland (82.11%)
Rock (79.44%)
Ecoregion (77.64%)
Pasture (76.98%)
Grass (76.20%)
Hill (75.14%)
Rural area (71.17%)
Working animal (69.03%)
Horse (67.90%)
Steppe (64.25%)
Labels (and confidence score):
Yellow (91.82%)
Water (91.00%)
Elephant (86.43%)
Elephants and Mammoths (80.01%)
Wildlife (72.86%)
Organism (71.52%)
African elephant (67.97%)
Wave (67.40%)
Labels (and confidence score):
Bird (99.60%)
Vertebrate (98.51%)
Beak (98.33%)
Seabird (71.23%)
Stork (65.82%)
Ciconiiformes (60.37%)
Water bird (55.88%)
Feather (50.94%)
Labels (and confidence score):
Formation (91.00%)
Rock (89.57%)
Fault (84.27%)
Badlands (84.22%)
Geology (84.12%)
Geological phenomenon (82.74%)
Bedrock (67.52%)
Dike (66.89%)
Outcrop (64.30%)
Intrusion (63.12%)
Labels (and confidence score):
Mountainous landforms (97.92%)
Mountain (97.04%)
White (96.29%)
Black (95.72%)
Atmospheric phenomenon (95.26%)
Mist (94.63%)
Sky (93.82%)
Black-and-white (91.53%)
Fog (89.18%)
Mountain range (88

Labels (and confidence score):
Sky (98.09%)
Night (95.72%)
Blue (92.70%)
Green (90.68%)
Light (89.30%)
Tree (86.01%)
Lighting (81.84%)
Water (80.73%)
Atmosphere (80.43%)
Cloud (76.76%)
Labels (and confidence score):
Wall (90.05%)
Street art (88.39%)
Art (85.11%)
Poster (76.33%)
Mural (73.28%)
Graffiti (63.80%)
Advertising (53.03%)
Labels (and confidence score):
People (94.50%)
Adaptation (79.12%)
Event (62.74%)
Tribe (60.90%)
Crowd (50.70%)
Labels (and confidence score):
People (95.36%)
Red (90.19%)
Child (81.40%)
Fun (76.85%)
Adaptation (74.08%)
Room (65.66%)
Leisure (63.88%)
Event (62.74%)
Photography (62.42%)
Vacation (54.60%)
Labels (and confidence score):
People (93.70%)
Water (91.00%)
Jumping (88.86%)
Fun (86.50%)
Child (84.02%)
Human (74.68%)
Happy (72.59%)
Play (69.64%)
Adaptation (66.97%)
Photography (62.42%)
Labels (and confidence score):
Vertebrate (98.51%)
Bird (98.37%)
Macaw (98.09%)
Parrot (94.51%)
Beak (89.26%)
Adaptation (79.12%)
Wing (77.76%)
Organism (76.72%)
Lovebird

...

Labels (and confidence score):
Sky (92.81%)
Atmospheric phenomenon (88.61%)
Atmosphere (82.49%)
Morning (82.10%)
Pc game (67.61%)
Digital compositing (67.32%)
Cloud (66.45%)
Evening (65.49%)
Landscape (65.04%)
Photography (62.42%)
Labels (and confidence score):
Sky (96.92%)
Natural landscape (96.60%)
Mountainous landforms (92.85%)
Flower (90.88%)
Mountain (90.83%)
Cloud (90.82%)
Natural environment (90.04%)
Wilderness (86.62%)
Atmospheric phenomenon (85.69%)
Wildflower (85.67%)
Labels (and confidence score):
California sea lion (87.47%)
Marine mammal (80.99%)
Fish (70.16%)
Vacation (60.00%)
World (53.14%)
Seal (50.02%)
Labels (and confidence score):
Geological phenomenon (88.41%)
Arctic (57.74%)
Marine mammal (57.29%)
Illustration (50.41%)
Labels (and confidence score):
Leisure (78.88%)
Fun (76.85%)
Vehicle (70.62%)
Vacation (66.44%)
Photography (62.42%)
Table (58.65%)
Boating (51.88%)
Travel (51.82%)
Tourism (51.14%)
Labels (and confidence score):
Vehicle (87.22%)
Car (82.42%)
Luxury 

Labels (and confidence score):
Ocean (93.63%)
Natural environment (92.02%)
Arctic ocean (91.91%)
Arctic (90.76%)
Water (80.73%)
Polar bear (80.60%)
Ice (80.50%)
Sea ice (79.93%)
Sea (75.85%)
Sky (73.26%)
Labels (and confidence score):
Bird (99.60%)
Vertebrate (98.51%)
Beak (98.33%)
Perching bird (80.47%)
Swallow (77.91%)
Songbird (76.46%)
Bluebird (70.54%)
Old World flycatcher (63.82%)
Wildlife (57.62%)
Tail (52.64%)
Labels (and confidence score):
Photograph (94.98%)
Beauty (88.24%)
Sitting (87.69%)
Sky (73.26%)
Fun (70.39%)
Photography (67.79%)
Dress (67.65%)
Furniture (61.48%)
Leisure (56.19%)
Smile (54.04%)
Labels (and confidence score):
Mountainous landforms (97.80%)
Mountain (97.38%)
Mountain range (92.65%)
Glacial landform (88.67%)
Geological phenomenon (87.52%)
Rock (85.98%)
Ridge (82.29%)
Alps (81.65%)
Arête (80.46%)
Massif (79.88%)
Labels (and confidence score):
Elephant (99.14%)
Elephants and Mammoths (98.09%)
African elephant (96.91%)
Indian elephant (95.95%)
Terrestrial ani

Labels (and confidence score):
Child (83.56%)
Transport (83.51%)
Passenger (74.81%)
Sitting (72.91%)
Adaptation (66.97%)
Room (65.66%)
Student (63.68%)
Labels (and confidence score):
Youth (82.78%)
Child (76.55%)
Adaptation (66.97%)
Tourism (60.85%)
Recreation (59.43%)
House (59.29%)
Leisure (56.19%)
Vacation (54.60%)
Dance (52.89%)
Style (52.46%)
Labels (and confidence score):
Sitting (83.21%)
Human (76.88%)
Photography (62.42%)
Black hair (56.98%)
Flash photography (55.41%)
Comfort (52.84%)
Labels (and confidence score):
Sitting (97.01%)
Pink (81.47%)
Adaptation (66.97%)
Shoe (56.94%)
Leisure (56.19%)
Vacation (54.60%)
Smile (54.04%)
Labels (and confidence score):
Public space (84.29%)
Community (80.28%)
Recreation (59.43%)
Playground (56.98%)
Team (52.32%)
Labels (and confidence score):
Organism (85.94%)
Eye (85.02%)
Close-up (84.62%)
Organ (80.63%)
Macro photography (79.86%)
Photography (67.79%)
Wildlife (67.62%)
Plant (66.16%)
Labels (and confidence score):
Balloon (57.04%)
Magent

Labels (and confidence score):
Wildlife (94.24%)
Silhouette (93.53%)
Deer (93.39%)
Sky (90.14%)
Elk (89.41%)
Sunset (82.69%)
Sunrise (69.25%)
Stock photography (65.44%)
Reindeer (53.84%)
Fawn (53.01%)
Labels (and confidence score):
Fish (99.47%)
Underwater (96.53%)
Shark (96.02%)
Marine biology (95.55%)
Great white shark (95.06%)
Cartilaginous fish (93.19%)
Tiger shark (88.61%)
Lamniformes (88.32%)
Carcharhiniformes (80.23%)
Lamnidae (79.92%)
Labels (and confidence score):
Tree (91.94%)
Wildlife (87.48%)
Sky (86.22%)
Atmospheric phenomenon (85.69%)
Morning (82.10%)
Branch (78.08%)
Sunlight (68.76%)
Sunrise (68.70%)
Backlighting (65.87%)
Plant (64.64%)
Labels (and confidence score):
Tree (99.44%)
Nature (96.55%)
Natural environment (94.51%)
Forest (93.67%)
Woody plant (93.52%)
Vegetation (91.24%)
Plant (90.74%)
Woodland (89.96%)
Nature reserve (89.65%)
Temperate broadleaf and mixed forest (89.44%)
Labels (and confidence score):
Tree (99.44%)
Woody plant (91.71%)
Branch (91.54%)
Trunk (9

Labels (and confidence score):
Town (85.18%)
Street (84.42%)
Alley (81.05%)
Infrastructure (75.79%)
Road (71.34%)
Architecture (65.45%)
Photography (62.42%)
Building (62.30%)
Temple (61.13%)
Tourism (56.89%)
Labels (and confidence score):
Polar bear (98.81%)
Bear (97.35%)
Arctic (97.07%)
Arctic ocean (93.28%)
Natural environment (92.87%)
Sky (92.13%)
Ocean (91.06%)
Ice (89.91%)
Polar ice cap (88.84%)
Sea ice (88.49%)
Labels (and confidence score):
Water (91.00%)
Town (82.94%)
Waterway (79.85%)
Tourism (71.09%)
Vehicle (63.02%)
Harbor (60.45%)
Sea (60.35%)
Travel (54.82%)
Vacation (54.60%)
Coast (53.17%)
Labels (and confidence score):
Nature (97.21%)
Atmospheric phenomenon (95.24%)
Mist (93.81%)
Sky (88.58%)
Natural landscape (88.55%)
Water (80.73%)
Tree (77.84%)
Atmosphere (77.52%)
Landscape (72.70%)
Terrain (70.44%)
Labels (and confidence score):
Tree (86.46%)
Tourism (81.04%)
Palm tree (80.61%)
Community (79.56%)
Adaptation (74.08%)
Arecales (71.95%)
Crowd (69.16%)
Leisure (68.10%)
T

In [24]:
api_df['Labels'] = ls_of_label_for_url
api_df.index = range(len(api_df.index))
api_df
# export_csv = api_df.to_csv (r'C:\Users\david\Pictures\UGCA\Final\Labeled_Instagram_df.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

,URL,Caption,Like Count,Comment Count,Labels
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268,"[Rays and skates, Fish, Underwater, Water, Man..."
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519,[Photography]
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491,"[Badlands, Mountainous landforms, Sky, Nature,..."
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717,"[Rock, Tourism, Landscape, National park]"
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @gerdludwig | Known for their elabora...,149658,394,"[People, Community, Yellow, Tradition, Event, ..."
...,...,...,...,...,...
495,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Ira Block @irablockphoto | A red-tail...,454383,1601,"[Bird, Vertebrate, Bird of prey, Beak, Falcon,..."
496,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nora Lorek @noralorek | Three years a...,122497,377,"[Fun, Tree, Vacation, Tourism, Leisure, Crowd]"
497,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,"Photo by Ami Vitale @amivitale | On Monday, @U...",493142,2411,"[Rhinoceros, Horn, White rhinoceros, Black rhi..."
498,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Paolo Woods and Gabriele Galimberti ...,331186,2307,"[Science, Medical assistant, Research]"


In [25]:
export_csv = api_df.to_csv (r'C:\Users\david\Pictures\UGCA\Final\Final_Labeled_Instagram_df.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path